# Accessing saved models

In this notebook, we will use the Python client to access and work with the AutoAI model you have saved in your project.

See documentation: 
- Python Client API: https://ibm-wml-api-pyclient.mybluemix.net/
- REST API https://cloud.ibm.com/apidocs/machine-learning

## Install the IBM Watson Machine Learning library

In [ ]:
!pip install ibm-watson-machine-learning -U

## Create a client connection and retrieve some environment information

In [ ]:
import os, json, requests, time
from ibm_watson_machine_learning import APIClient

credentials_url = os.getenv('RUNTIME_ENV_APSX_URL')
user_id = os.getenv('USER_ID')
project_id = os.getenv('PROJECT_ID')
project_name = os.getenv('PROJECT_NAME')
token = os.environ['USER_ACCESS_TOKEN']

In [ ]:
WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "openshift",
   "url": credentials_url,
   "version": "3.5"
}

client = APIClient(WML_CREDENTIALS)
print("Client version:", client.version)

In [ ]:
space_name = user_id + '_space'

print("Space name: " + space_name + ", project id: " + project_id)

## Create a WML space for this specific user

We will create a deployment space for the model with the space name we specified above. If the space already exists, we will use the existing space.

In [ ]:
spaces = client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
        print('Using existing space')
if space_id is None:
    print('Creating new space')
    space_id = client.spaces.store(
        meta_props={client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["id"]

## List the available spaces

In [ ]:
client.spaces.list()

## Get the model from the project

Next, we set the client to use the project that contains the model. We can then get the details of the model. The code below assumes that the AutoAI model is the only one listed in the project.

In [ ]:
client.set.default_project(project_id)

In [ ]:
model_details = client.repository.get_model_details()
myModel_details = model_details['resources'][0]

model_uid = myModel_details['metadata']['id']
model_name = myModel_details['metadata']['name']
model_href = client.repository.get_model_href(myModel_details)
print(model_name + " guid: " + model_uid)

## Get the training information

The training ID is necessary to deploy the model. Once again, the code below assumes that the AutoAI model is the only one in the project, so we can use the first training from the list.

In [ ]:
training_id = client.training.get_details()["resources"][0]["metadata"]["id"]

## Create the web service

Using the model and training info, our WML credentials, and the associated project and deployment space, we can create a web service for our deployed model. The code below uses the first pipeline generated by AutoAI, but can easily be modified to use different pipelines.

In [ ]:
from ibm_watson_machine_learning.deployment import WebService

service = WebService(source_wml_credentials=WML_CREDENTIALS,
                     target_wml_credentials=WML_CREDENTIALS,
                     source_project_id=project_id,
                     target_space_id=space_id)

In [ ]:
service.create(experiment_run_id=training_id,
               model='Pipeline_1',
               deployment_name=model_name + '_deployed')

## Get information for the deployed model

Next, we can access the deployment space and get the details for the deployed model so that we can send requests to it.

In [ ]:
client.set.default_space(space_id)

In [ ]:
deployment_id = None
for deployment in client.deployments.get_details()['resources']:
    if deployment["metadata"]["name"] == model_name + '_deployed':
        deployment_id = deployment["metadata"]["id"]
print("Deployment ID:", deployment_id)

## Send a request to the deployed model

We can get the format for the request from the model details.

In [ ]:
fieldnames = [x['name'] for x in myModel_details['entity']['schemas']['input'][0]['fields']] 

output = "["
for field in fieldnames:
    output += '"' + field + '",'
output += ']'
output

In [ ]:
# Execute the model
scoring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA:
                      [{
                         'fields': fieldnames,
                         'values': [[1, 45970, 'YES', 'Owner Occupier', 11, 7, 1, 579, 0, 9280, 132000, 'L100'],
                                    [2, 43823, 'YES', 'Living with parents/guardian', 14, 1, 2, 2245, 1, 9795, 175000, 'L130']
                                   ]
                        }]
                      }
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

## Remove the deployed model

Active deployed models will consume resources. We can easily remove the deployment using the API.

In [ ]:
client.deployments.delete(deployment_id)